In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time

In [2]:
def browserexec():
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

In [ ]:
def scrape():
    browser = browserexec()

    # Scrape latest title and paragraph
    
    browser.visit('https://redplanetscience.com/')
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')


    ### Get first news title and paragraph
    quote = soup.find('div', class_='content_title').text
    summary = soup.find('div', class_='article_teaser_body').text

    
    # Scrape for feautured image
    
    browser.visit('https://spaceimages-mars.com/')
    html_img = browser.html
    soup_img = BeautifulSoup(html_img, 'html.parser')

    ### Get image source
    img_link = soup_img.find('img', class_='headerimage fade-in')
    img_link = img_link['src']

    ### Get link of image source
    featured_img_link = str(url_img) + str(img_link)


    # Scrape for Mars Facts
    
    browser.visit('https://galaxyfacts-mars.com')
    
    ### Read table and print
    df_list = pd.read_html('https://galaxyfacts-mars.com')
    table = df_list[1].head(7)
    table = table.rename(columns={0: "Fact", 1: "Value"})
    
    ### table to html
    html_table = table.to_html()
    


    # MARS Hemispheres
    html_hem = browser.html
    soup_hem = BeautifulSoup(html_hem, 'html.parser')

    #finding section that contains information about each hemisphere
    hem_data = soup_hem.find_all('div', class_="collapsible results")
    names = hem_data[0].find_all('h3')

    name_list = []

    # Loop through each name
    for name in names: 
        name_list.append(name.text)

    # loop through each image link

    links = hem_data[0].find_all('a')
    links_dict = []

    for link in links:
        if (link.img):
            url_img = url_hem + link['href']
            links_dict.append(url_img)

    # from each of the links, grab the photo

    images = []

    for url in links_dict:

        browser.visit(url)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')

        # find image and append
        image = soup.find_all('img', class_='wide-image')
        image_src = image[0]['src']
        good_img_link = url_hem + image_src
        images.append(good_img_link)
    
    #create dataframe to add titles
    title_creator = pd.DataFrame({'Name':name_list,"Img_Link":images})
    # transfer df to dictionary
    dictionary = title_creator.to_dict()

    
    
    
    # All data
    all_data = {
        "Title": quote,
        "Paragraph": summary,
        "Featured_image": featured_img_link,
        "Facts": table,
        "hemispheres": dictionary
    }

    # Close the browser after scraping
    browser.quit()

    # Return results
    return all_data
